In [1]:
import pandas as pd
from Skeleton import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv

gui = nimble.NimbleGUI()
gui.serve(8080)
# sudo lsof -iTCP:8080 -sTCP:LISTEN

s12 = Skeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY12.xml')
# Rajagopal
# s = DynamicSkeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml')
# BSM
s = DynamicSkeleton(config='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml', \
    osim_file='/media/rmhri/mrom/data/skel_models_v1.1/bsm.osim',\
    max_velocity=10)

GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080
Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0


/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/Skeleton/DynamicSkeleton.py:200: RuntimeWarning: invalid value encountered in add
  s_avg = (self.q_l + self.q_u) / 2


In [2]:
# s._nimble.getJointWorldPositionsJacobianWrtBodyScales(s.joints)[:,0:72:3]

In [3]:
# Read the file in Shafagh format and collapse the header into one line

filename = "tmp/L266/23-10-28_Shafagh_free_world.csv"
# filename = "tmp/L266/23-10-28_Shafagh_free.csv"
out_header = ['']*465
out_file = []
with open(filename, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for i,row in enumerate(spamreader):
        if i > 1 and i < 7 and i != 4:
            for j in range(465):
                if row[j] == "''" or out_header[j] == '':
                    out_header[j] += row[j]
                else:
                    out_header[j] += "." + row[j]
        if i > 6:
            out_file.append(row)

with open(filename.replace(".csv",".oneheader.csv"), 'w') as f:
    writer = csv.writer(f,delimiter=',')
    writer.writerow(out_header)
    writer.writerows(out_file)


In [4]:
# Read the modified version of the csv in pandas
filename = "tmp/L266/23-10-28_Shafagh_free_world.oneheader.csv"
df = pd.read_csv(filename,delimiter=',')

# header = list(df.columns)

# bones = []

# for h in range(df.shape[1]):
#     if "Bone.Skeleton:" in header[h] and "Position" in header[h]:
#         bones.append(header[h].split('.')[1].split(":")[1])
# print(list(set(bones)))        
        
# ['RThumb2', 'LPinky1', 'LMiddle2', 'LIndex2', 'RIndex2', 'RMiddle1', \
#     'LPinky3', 'RPinky1', 'RPinky3', 'RMiddle3', 'Neck', 'RRing3', 'LMiddle1', \
#         'RShoulder', 'LHand', 'RIndex1', 'LIndex1', 'LIndex3', 'LFArm', 'RHand', \
#             'LShoulder', 'LRing1', 'LRing3', 'RThumb1', 'RThumb3', 'Ab', 'LUArm', \
#                 'RIndex3', 'RPinky2', 'Skeleton', 'LThumb1', 'LThumb3', 'LPinky2', \
#                     'Head', 'RMiddle2', 'LRing2', 'RRing2', 'RRing1', 'RFArm', 'LThumb2', \
#                         'RUArm', 'Chest', 'LMiddle3']

s.body_dict["thorax"] = "LClavicle"

kps_optitrack = ['LHand','LFArm','RHand','LUArm','RFArm','RUArm']
kps =  ['', 'LHand', 'RHip', 'RUArm',  'LFArm', 'LHip', 'RFArm', 'RHand', '', 'LUArm', '', '']
Q = []
P = []
P_target = []
times = []
s.prob = None
s.reset()
for t in range(0,df.shape[0]):
    target = []
    #np.array([np.array(df.loc[t,["Bone.Skeleton:"+l+".Position.X","Bone.Skeleton:"+l+".Position.Y","Bone.Skeleton:"+l+".Position.Z"]].to_numpy()) for l in kps]).squeeze()/1000
    for kp in kps:
        if kp == '':
            target.append([np.nan,np.nan,np.nan])
        elif kp == 'LHip':
            LASI = np.array(df.loc[t,["Marker.Skeleton:LASI.Position.X","Marker.Skeleton:LASI.Position.Y","Marker.Skeleton:LASI.Position.Z"]].to_numpy()).squeeze()/1000
            LPSI = np.array(df.loc[t,["Marker.Skeleton:LPSI.Position.X","Marker.Skeleton:LPSI.Position.Y","Marker.Skeleton:LPSI.Position.Z"]].to_numpy()).squeeze()/1000
            target.append((LASI+LPSI) / 2)        
        elif kp == 'RHip':
            RASI = np.array(df.loc[t,["Marker.Skeleton:RASI.Position.X","Marker.Skeleton:RASI.Position.Y","Marker.Skeleton:RASI.Position.Z"]].to_numpy()).squeeze()/1000
            RPSI = np.array(df.loc[t,["Marker.Skeleton:RPSI.Position.X","Marker.Skeleton:RPSI.Position.Y","Marker.Skeleton:RPSI.Position.Z"]].to_numpy()).squeeze()/1000
            target.append((RASI+RPSI) / 2)        
            
        else:
            target.append(np.array(df.loc[t,["Bone.Skeleton:"+kp+".Position.X","Bone.Skeleton:"+kp+".Position.Y","Bone.Skeleton:"+kp+".Position.Z"]].to_numpy()).squeeze()/1000)
    
    target = np.array(target)
    
    # res = np.dot(R,target.reshape(-1,3).transpose()*1000) + T
    Rx = np.array([
        [1,0,0],
        [0,0,1],
        [0,-1,0]
    ])
    res = np.dot(Rx,target.reshape(-1,3).transpose()*1000)
    target = res.transpose()/1000
    
    # print(s.kps)
        
    s12.load_from_numpy(target.reshape(-1,3),s.kps)
    s.load_from_BODY12(s12)
    
    
    
    # s.estimate_confidence()
    if t == 0:
        s.exact_scale()
    
    # s.gdIK(1000)
    # start = time.time()
    s.qpIK(1000,0.01,precision=0.0001)
    # # s.filter(dt=0.02)    
    # stop = time.time()
    
    gui.nativeAPI().renderSkeleton(s._nimble)
    
    pos = s._nimble.getJointWorldPositions(s.joints)
    pos = s.correct(pos)

    target = target.reshape(-1,1)
    for i in range(0,int(target.shape[0]/3)):
        gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.01,0.01,0.01]),pos=target[3*i:3*i+3],color=np.array([0,255,0,0.8]))
    
    for i in range(0,int(pos.shape[0]/3)):
        gui.nativeAPI().createSphere(key=s.joints[i].getName(), radii=np.array([0.01,0.01,0.01]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,1]))
    Q.append(s._nimble.getPositions())
    P.append(s.to_numpy())
    P_target.append(target)
    s.reset_history()
    # break
    # time.sleep(0.01)
    # break
Q = np.array(Q)
P = np.array(P)
P_target = np.array(P_target)
# print(np.mean(times))

(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,)
(72,

127.0.0.1 - - [02/May/2024 15:29:36] "GET / HTTP/1.1" 304 -


In [ ]:

import matplotlib.pyplot as plt
angles = []
for i in range(s._nimble.getNumJoints()):
    j = s._nimble.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getName()+'_'+j.getDofName(k))

i = 34
plt.title(angles[i]+" ("+str(i)+")" )
plt.plot(np.array(Q)[:,i]*180/np.pi)
# plt.plot(np.array(Q_qp10r)[:,i]*180/np.pi,label="g25 and +-10 rad/s")
# plt.plot(np.array(Q_qp2r)[:,i]*180/np.pi,label="g25 and +-2 rad/s")
# plt.legend()
plt.show()